### Workflow of a Linear Regression model

In [36]:
# Import PyTorch and matplotlib
import torch
from torch import nn # nn contains all of PyTorch's building blocks for neural networks
import matplotlib.pyplot as plt

# Check PyTorch version
torch.__version__

'2.8.0'

In [37]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

## 1.1 DATA

In [38]:
# Create some data using Linear Regression formula of y = mX + c 
weight = 0.7
bias = 0.3

# Create range values
start = 0
end = 1
step = 0.02

# Create X and y (features and labels)
X = torch.arange(start, end, step).unsqueeze(dim=1) # without unsqueeze, errors will pop up
y = weight * X + bias

print(f'X:{X[:10]}, y: {y[:10]}')

X:tensor([[0.0000],
        [0.0200],
        [0.0400],
        [0.0600],
        [0.0800],
        [0.1000],
        [0.1200],
        [0.1400],
        [0.1600],
        [0.1800]]), y: tensor([[0.3000],
        [0.3140],
        [0.3280],
        [0.3420],
        [0.3560],
        [0.3700],
        [0.3840],
        [0.3980],
        [0.4120],
        [0.4260]])


In [39]:
# Split data
train_split = int(0.8 * len(X)) # 80% of data used for training set, 20% for testing 
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

(40, 40, 10, 10)

## 1.2 BUILD MODEL

In [40]:
# Building a Pytorch Linear model 
class LinearRegressionModel_V2(nn.Module): 
    def __init__(self):
        super().__init__()
        # use nn.Linear() for creating the model parameters
        # also called: linear transform, probing layer, fully connected layer, dense layer
        self.linear_layer = nn.Linear(in_features=1, out_features=1)
    
    def forward(self, x):
        return self.linear_layer(x)

# class LinearRegressionModel(nn.Module): # <- almost everything in PyTorch is a nn.Module (think of this as neural network lego blocks)
#     def __init__(self):
#         super().__init__() 
#         self.weights = nn.Parameter(torch.randn(1, # <- start with random weights (this will get adjusted as the model learns)
#                                                 dtype=torch.float), # <- PyTorch loves float32 by default
#                                    requires_grad=True) # <- can we update this value with gradient descent?)

#         self.bias = nn.Parameter(torch.randn(1, # <- start with random bias (this will get adjusted as the model learns)
#                                             dtype=torch.float), # <- PyTorch loves float32 by default
#                                 requires_grad=True) # <- can we update this value with gradient descent?))

#     # Forward defines the computation in the model
#     def forward(self, x: torch.Tensor) -> torch.Tensor: # <- "x" is the input data (e.g. training/testing features)
#         return self.weights * x + self.bias # <- this is the linear regression formula (y = m*x + b)



In [41]:
# Set manual seed
torch.manual_seed(42)
model_1 = LinearRegressionModel_V2()
model_1, model_1.state_dict()

(LinearRegressionModel_V2(
   (linear_layer): Linear(in_features=1, out_features=1, bias=True)
 ),
 OrderedDict([('linear_layer.weight', tensor([[0.7645]])),
              ('linear_layer.bias', tensor([0.8300]))]))

In [42]:
# Create the loss function
loss_fn = nn.L1Loss() # MAE loss is same as L1Loss

# Create the optimizer
optimizer = torch.optim.SGD(params=model_1.parameters(), # parameters of target model to optimize
                            lr=0.001) # learning rate (how much the optimizer should change parameters at each step, higher=more (less stable), lower=less (might take a long time))

## 1.3 TRAINING LOOP

In [ ]:
# an epoch is one loop through the data... Set the number of epochs (hyperparameter because we set the value)
epochs = 200

### TRAINING 

# 0. Loop through the data
for epoch in range(epochs):

    # set model in training mode (this is the default state of a model)
    # it sets all paramters that require gradient descent to require gradients
    model_1.train()

    # 1. Forward pass on train data using the forward() method inside 
    y_pred = model_1(X_train)

    # 2. Calculate the loss (how different are our models predictions to the ground truth)
    loss = loss_fn(y_pred, y_train)

    # 3. Zero grad of the optimizer - not for first time; 2nd epoch onwards - 
    # we need to reset the optimiser gradients every epoch; start fresh each forward pass
    optimizer.zero_grad()

    # 4. Loss backwards - perform backpropagation on the loss wrt the parameters of the model (nn.Parameter mein wherever it is requires_grad = True)
    loss.backward()

    # 5. Progress the optimizer (perform gradient descent)
    optimizer.step()


    ### Testing
    # turns off the dfferent settings in the model which are not needed for evaluation/testing (batch norm layers/ dropout)
    model_1.eval()

    # turns off gradient descent
    with torch.inference_mode():
      # 1. Forward pass on test data
      test_pred = model_1(X_test)
  
      # 2. Caculate loss on test data
      test_loss = loss_fn(test_pred, y_test.type(torch.float)) # predictions come in torch.float datatype, so comparisons need to be done with tensors of the same type

      # Print out what's happening
      if epoch % 10 == 0:
            print(f"Epoch: {epoch} | MAE Train Loss: {loss} | MAE Test Loss: {test_loss} ")

Epoch: 0 | MAE Train Loss: 0.5551779866218567 | MAE Test Loss: 0.5861001014709473 
Epoch: 10 | MAE Train Loss: 0.543657124042511 | MAE Test Loss: 0.5726292729377747 
Epoch: 20 | MAE Train Loss: 0.5321362614631653 | MAE Test Loss: 0.559158444404602 
Epoch: 30 | MAE Train Loss: 0.5206154584884644 | MAE Test Loss: 0.545687735080719 
Epoch: 40 | MAE Train Loss: 0.5090945959091187 | MAE Test Loss: 0.5322169065475464 
Epoch: 50 | MAE Train Loss: 0.49757370352745056 | MAE Test Loss: 0.5187460780143738 
Epoch: 60 | MAE Train Loss: 0.48605290055274963 | MAE Test Loss: 0.5052752494812012 
Epoch: 70 | MAE Train Loss: 0.47453203797340393 | MAE Test Loss: 0.49180445075035095 
Epoch: 80 | MAE Train Loss: 0.4630111753940582 | MAE Test Loss: 0.47833362221717834 
Epoch: 90 | MAE Train Loss: 0.4514903128147125 | MAE Test Loss: 0.4648628234863281 
Epoch: 100 | MAE Train Loss: 0.4399694800376892 | MAE Test Loss: 0.4513920247554779 
Epoch: 110 | MAE Train Loss: 0.4284486174583435 | MAE Test Loss: 0.4379211

In [44]:
model_1.state_dict()

OrderedDict([('linear_layer.weight', tensor([[0.6865]])),
             ('linear_layer.bias', tensor([0.6300]))])

In [45]:
weight, bias

(0.7, 0.3)